In [16]:
import taxcalc as tc
import pandas as pd
import numpy as np
import copy

In [17]:
# Dep_credit: nonrefundable portion
# c07220: refundable portion, not yet calculated as ACTC
# c11070: portion actually refunded
# TODO: Create a function that takes a subpopulation and spits out total change in revenue, money going to CTC, eitc, per person, etc.

def taxdf(calc, namelist, varlist):
    calcu=copy.deepcopy(calc)
    tdf=pd.DataFrame()
    for v in range (0, len(namelist)):
        tdf[namelist[v]]=getattr(calcu.records,varlist[v])
    return tdf

In [18]:
def percentile(df, percentile, var):
    df = df.sort_values(var, axis = 0)
    df['cumsum'] = np.cumsum(df['s006'])
    return df[df['cumsum'] <= (percentile/100) * df['cumsum'].max()].iloc[-1][var]

In [19]:
reform_noctc = {2018:{}}

reform_noctc[2018]['_CTC_c'] = [1000]
reform_noctc[2018]['_DependentCredit_Child_c'] = [0]
reform_noctc[2018]['_ACTC_Income_thd'] = [3000]
reform_noctc[2018]['_CTC_ps'] = [[75000, 110000, 55000, 75000, 75000]]

# Create calculator for TCJA Reconciliation plan, minus refundable child tax credit

pol_noctc = tc.Policy()
pol_noctc.implement_reform(reform_noctc)

calc = tc.Calculator(policy = pol_noctc, records = tc.Records())
calc.advance_to_year(2018)
calc.calc_all()

# Create calculator for TCJA Reconciliation plan EXPANSION

calc1 = tc.Calculator(records = tc.Records(), policy = tc.Policy())
calc1.advance_to_year(2018)
calc1.calc_all()

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [20]:
nlist = ['c00100', 'standard', 'c04470', 'iitax','combined','payrolltax','s006','MARS', 'n24', 'XTOT', 'c11070', 'c09600', 'refund', 'eitc']
calcdf = taxdf(calc=calc, namelist = nlist, varlist = nlist)
calcdf1 = taxdf(calc=calc1, namelist = nlist, varlist = nlist)

In [21]:
# GET TOTAL MONEY SPENT ON CHILD TAX REFUNDS
# MAKE SINGLE PARENT DUMMY ARRAY
def costdiff(no_expansion, expansion, subpop):
    expansion['w_iitaxdiff'] = (expansion['iitax'] - no_expansion['iitax'])*no_expansion['s006']
    expansion['w_c11070'] = expansion['c11070'] * no_expansion['s006']
    expansion['w_eitc'] = expansion['eitc'] * no_expansion['s006']
    expansion_sp = copy.deepcopy(expansion[subpop])
    singleparent = ((expansion_sp['MARS'] == 1) | (expansion_sp['MARS'] == 4)) & (expansion_sp['n24']  >  0)
    num_single_parents = expansion_sp[singleparent]['s006'].sum()
    total = expansion_sp['s006'].sum()
    print("Number of Beneficiaries in Group that are Single or Head of Household with Kids: " +
          '{:,}'.format(round(num_single_parents)))
    print("Number of Beneficiaries in Group: " +
          '{:,}'.format(round(total)))
    print("Amount of Revenue Going towards Taxpayers in Group: " + 
          '{:,}'.format(round(expansion_sp['w_iitaxdiff'].sum())))
    print('Amount of Revenue Going Towards Single Parent Beneficiaries: ' +
          '{:,}'.format(round(expansion_sp[singleparent]['w_iitaxdiff'].sum())))
    print('Total ACTC Benefits Claimed by Group: ' +
          '{:,}'.format(round(expansion[subpop]['w_c11070'].sum())))
    print('Total ACTC Benefits Claimed by Single Parents in Group: ' +
          '{:,}'.format(round(expansion_sp[singleparent]['w_c11070'].sum())))
    print('Total EITC Benefits Claimed by Group: ' +
          '{:,}'.format(round(expansion[subpop]['w_eitc'].sum())))
    print('Total EITC Benefits Claimed by Single Parents in Group: ' +
          '{:,}'.format(round(expansion_sp[singleparent]['w_eitc'].sum())))

In [22]:
# CHECK IF REFUND RECEIVING BENEFICIARIES ARE FOR CURRENT LAW, TCJA, or TCJA without CTC expansion
pop0 = [([True] * len(calcdf)), 'All Tax Filers']
pop1 = [(calcdf['iitax'] < 0), 'Refund-Receiving Beneficiaries']
pop2 = [((calcdf['iitax'] < 0) & (calcdf['c00100'].between(5000, 10000))), 'Refund-Receiving Beneficiaries, AGI of 5-10 Thousand']
pop3 = [((calcdf['iitax'] < 0) & (calcdf['c00100'].between(10000, 15000))), 'Refund-Receiving Beneficiaries, AGI of 10-15 Thousand']
pop4 = [((calcdf['iitax'] < 0) & (calcdf['c00100'].between(15000, 20000))), 'Refund-Receiving Beneficiaries, AGI of 15-20 Thousand']
pop5 = [((calcdf['iitax'] < 0) & (calcdf['c00100'].between(20000, 25000))), 'Refund-Receiving Beneficiaries, AGI of 20-25 Thousand']
pop6 = [((calcdf['iitax'] < 0) & (calcdf['c00100'].between(25000, 30000))), 'Refund-Receiving Beneficiaries, AGI of 25-30 Thousand']

for x in [pop0, pop1, pop2, pop3, pop4, pop5, pop6]:
    print(x[1])
    costdiff(calcdf, calcdf1, x[0])
    print('\n')

All Tax Filers
Number of Beneficiaries in Group that are Single or Head of Household with Kids: 20,068,092.0
Number of Beneficiaries in Group: 172,859,804.0
Amount of Revenue Going towards Taxpayers in Group: -66,904,066,744.0
Amount of Revenue Going Towards Single Parent Beneficiaries: -18,617,252,898.0
Total ACTC Benefits Claimed by Group: 36,820,164,309.0
Total ACTC Benefits Claimed by Single Parents in Group: 23,806,212,323.0
Total EITC Benefits Claimed by Group: 65,815,190,480.0
Total EITC Benefits Claimed by Single Parents in Group: 41,343,595,450.0


Refund-Receiving Beneficiaries
Number of Beneficiaries in Group that are Single or Head of Household with Kids: 14,210,254.0
Number of Beneficiaries in Group: 32,356,014.0
Amount of Revenue Going towards Taxpayers in Group: -15,338,551,900.0
Amount of Revenue Going Towards Single Parent Beneficiaries: -10,001,135,531.0
Total ACTC Benefits Claimed by Group: 34,250,998,932.0
Total ACTC Benefits Claimed by Single Parents in Group: 22,7

In [23]:
percentile(calcdf[pop1[0]], percentile=50, var = 'c00100')

16276.218704446032